# Compute potential intensity

```
conda create -n potint -y netcdf4 xarray=0.16.2 numpy numba ipykernel                           # don't add matplotlib after xarray - broke installation
conda activate potint
pip install tcpypi
```

In [1]:
# kernel: potint
import xarray as xr, os, glob, re
from datetime import datetime
import warnings; warnings.filterwarnings("ignore", category = FutureWarning)

from tcpyPI import pi
from tcpyPI.utilities import *

# CMIP6

In [4]:
fpath = "/rds/general/user/cb2714/home/00_WWA_project_folder/ephemeral/potential-intensity/"

# list models with available data
mlist = list(set([fp.split("_")[5] for fp in glob.glob(fpath+"tos-rg/*.nc") if not "Veg" in fp]))
pi_list = list(set([re.sub(".nc","",fp.split("_")[-1]) for fp in glob.glob(fpath+"pi/*.nc") if not "Veg" in fp]))
mlist = [m for m in mlist if not m in pi_list and not "EC-Earth" in m]

In [5]:
mlist# = ["MIROC6", "CMCC-CM2-SR5"]

[]

In [ ]:
for mdl in mlist:
        
    print(mdl)
    
    new_fnm = fpath+"pi/pi_"+mdl+".nc"
    if os.path.exists(new_fnm): continue
    
    # load all the variables
    tos = xr.concat([xr.open_dataset(fnm) for fnm in sorted(glob.glob(fpath+"tos-rg/*_"+mdl+"*.nc"))], "time")
    hus,psl,ta = [xr.concat([xr.open_dataset(fnm) for fnm in sorted(glob.glob(fpath+varnm+"/*_"+mdl+"*.nc"))], "time") for varnm in ["hus","psl","ta"]]
    
    # merge into single DF
    ds = xr.merge([tos, psl, ta, hus]).rename(plev = "p", ta = "t", hus = "q", psl = "msl")
    
    print("  Data loaded:   "+datetime.now().time().strftime("%H:%M:%S"))
    
    # calculate the potential intensity (may take a v long time - up to 3hrs for 200 years)
    vmax, pmin, ifl, t0, otl = xr.apply_ufunc(
        pi,
        ds['sst'], ds['msl'], ds['p'], ds['t'], ds['q'],
        kwargs=dict(CKCD=0.9, ascent_flag=0, diss_flag=1, ptop=50, miss_handle=1),  # used default value of CKCD = 0.9
        input_core_dims=[
            [], [], ['p', ], ['p', ], ['p', ],
        ],
        output_core_dims=[
            [], [], [], [], []
        ],
        vectorize=True
    )
    
    print("  PI calculated: "+datetime.now().time().strftime("%H:%M:%S"))
    
    # store the result in an xarray data structure
    ds_out = xr.Dataset({
        'vmax': vmax, 
        'pmin': pmin,
        'ifl': ifl,
        't0': t0,
        'otl': otl,
        })
    
    ds_out.to_netcdf(new_fnm)
    
    print("  Data saved:    "+datetime.now().time().strftime("%H:%M:%S"))

NESM3


## For models saved at yearly resolution
(takes too long to cat individual files together & run)

In [2]:
mdl = "EC-Earth3"

In [5]:
fl = sorted(glob.glob(fpath+"tos-rg/*_"+mdl+"*.nc"))

In [ ]:
missing_years = []
for fnm in fl:
    
    print(fnm[-20:-3], end = ": ")
    new_fnm = fpath+"pi/"+mdl+"/pi_"+mdl+"_"+fnm[-20:]
    if os.path.exists(new_fnm): 
        print("complete")
        continue
        
    tos = xr.open_dataset(fnm)
    
    if any([len(glob.glob(fpath+varnm+"/*"+fnm[-20:])) == 0 for varnm in ["hus", "psl", "ta"]]):
        print("missing")
        missing_years.append(fnm[-20:])
        continue
    
    hus, psl, ta = [xr.open_dataset(glob.glob(fpath+varnm+"/*"+fnm[-20:])[0]) for varnm in ["hus", "psl", "ta"]]
    
    # merge into single DF
    ds = xr.merge([tos, psl, ta, hus]).rename(plev = "p", ta = "t", hus = "q", psl = "msl")
    
    # calculate the potential intensity (may take a v long time - up to 3hrs for 200 years)
    vmax, pmin, ifl, t0, otl = xr.apply_ufunc(
        pi,
        ds['sst'], ds['msl'], ds['p'], ds['t'], ds['q'],
        kwargs=dict(CKCD=0.9, ascent_flag=0, diss_flag=1, ptop=50, miss_handle=1),  # used default value of CKCD = 0.9
        input_core_dims=[
            [], [], ['p', ], ['p', ], ['p', ],
        ],
        output_core_dims=[
            [], [], [], [], []
        ],
        vectorize=True
    )
    
    # store the result in an xarray data structure
    ds_out = xr.Dataset({
        'vmax': vmax, 
        'pmin': pmin,
        'ifl': ifl,
        't0': t0,
        'otl': otl,
        })

    ds_out.to_netcdf(new_fnm)
    print("complete")
clear_output()
print("Done.")
print(missing_years)

18500101-18501231: complete
18510101-18511231: 

In [ ]:
# compile the individual files
pi = xr.concat([xr.open_dataset(fnm) for fnm in sorted(glob.glob(fpath+"pi/"+mdl+"/*.nc"))], "time")
pi.to_netcdf(fpath+"pi/pi_"+mdl+".nc")

## PI from monthly means

In [19]:
fpath = "/rds/general/user/cb2714/home/00_WWA_project_folder/ephemeral/potential-intensity/monthly_vars/"

# list models with available data
mlist = ["_".join(fnm[:-3].split("_")[-2:]) for fnm in glob.glob(fpath+"psl*")]
pi_list = ["_".join(fnm[:-3].split("_")[-2:]) for fnm in glob.glob(fpath+"mpi*")]
mlist = [m for m in mlist if not m in pi_list]

In [22]:
for mdl in mlist:
        
    print(mdl)
    
    new_fnm = fpath+"mpi_"+mdl+".nc"
    if os.path.exists(new_fnm): continue
    
    ds = xr.merge([xr.open_dataset(fpath+varnm+"-monthly_"+mdl+".nc") for varnm in ["hus","psl","ta", "tos-rg"]])
    ds = ds.rename(plev = "p", ta = "t", hus = "q", psl = "msl")
    
    # calculate the potential intensity
    vmax, pmin, ifl, t0, otl = xr.apply_ufunc(
        pi,
        ds['sst'], ds['msl'], ds['p'], ds['t'], ds['q'],
        kwargs=dict(CKCD=0.9, ascent_flag=0, diss_flag=1, ptop=50, miss_handle=1),  # used default value of CKCD = 0.9
        input_core_dims=[
            [], [], ['p', ], ['p', ], ['p', ],
        ],
        output_core_dims=[
            [], [], [], [], []
        ],
        vectorize=True
    )
        
    # store the result in an xarray data structure
    ds_out = xr.Dataset({
        'vmax': vmax, 
        'pmin': pmin,
        'ifl': ifl,
        't0': t0,
        'otl': otl,
        })
    
    ds_out.to_netcdf(new_fnm)

MRI-ESM2-0_r1i1p1f1
EC-Earth3-Veg-LR_r1i1p1f1
IPSL-CM6A-LR_r1i1p1f1
NorESM2-LM_r1i1p1f1
EC-Earth3_r1i1p1f1
CanESM5_r1i1p1f1
MIROC6_r1i1p1f1
CMCC-ESM2_r1i1p1f1
EC-Earth3-CC_r1i1p1f1
MPI-ESM1-2-LR_r1i1p1f1
MPI-ESM1-2-HR_r1i1p1f1
EC-Earth3-Veg_r1i1p1f1
NorESM2-MM_r1i1p1f1


---
# ERA5

In [2]:
fpath = "data/era5_pi/"

In [3]:
for y in [2024]:
    print(y)
    
    new_fnm = fpath+"pi_era5-monthly_"+str(y)+".nc"
#     if os.path.exists(new_fnm): continue
    
    # load all the variables
    ds = xr.open_dataset("data/era5_pi/pi-vars-monthly_era5_2024.nc").rename(pressure_level = "p", latitude = "lat", longitude = "lon")

    # calculate the potential intensity (may take a v long time - up to 3hrs for 200 years)
    vmax, pmin, ifl, t0, otl = xr.apply_ufunc(
        pi,
        ds['sst'], ds['msl'], ds['p'], ds['t'], ds['q'],
        kwargs=dict(CKCD=0.9, ascent_flag=0, diss_flag=1, ptop=50, miss_handle=1),  # used default value of CKCD = 0.9
        input_core_dims=[
            [], [], ['p', ], ['p', ], ['p', ],
        ],
        output_core_dims=[
            [], [], [], [], []
        ],
        vectorize=True
    )
    
    # store the result in an xarray data structure
    ds_out = xr.Dataset({
        'vmax': vmax, 
        'pmin': pmin,
        'ifl': ifl,
        't0': t0,
        'otl': otl,
        })

    ds_out.to_netcdf(new_fnm)

2024


## ERA5 from monthly downloads

In [2]:
ds = xr.open_dataset("era5_monthly_pi-vars.nc")

In [4]:
# calculate the potential intensity
vmax_msl, pmin, ifl, t0, otl = xr.apply_ufunc(
    pi,
    ds['sst'], ds['msl'], ds['p'], ds['t'], ds['q'],
    input_core_dims=[
        [], [], ['p', ], ['p', ], ['p', ],
    ],
    output_core_dims=[
        [], [], [], [], []
    ],
    vectorize=True
)

In [5]:
vmax_sp, pmin, ifl, t0, otl = xr.apply_ufunc(
    pi,
    ds['sst'], ds['sp'], ds['p'], ds['t'], ds['q'],
    input_core_dims=[
        [], [], ['p', ], ['p', ], ['p', ],
    ],
    output_core_dims=[
        [], [], [], [], []
    ],
    vectorize=True
)

In [6]:
xr.Dataset({'vmax_sp': vmax_sp, 'vmax_msl': vmax_msl}).to_netcdf("era5_monthly_vmax.nc")